# HW3 103011110 吳幸儒

## import library

In [1]:
import time
import numpy as np
import math
%matplotlib inline
import matplotlib.pylab as plt

# MLP function

In [14]:
class MLP(object):
    '''Multiple Layer Perceptron'''
    
    def __init__(self, Layers=(2, 5, 3), BatchSize = 4):
        self.bs = BatchSize
        self.net = [dict() for i in range(len(Layers))] # A list of fully-connected Layers
       
        self.net[0]['i'] = np.zeros((Layers[0], self.bs), dtype='float64')
        self.net[0]['z'] = np.zeros((Layers[0], self.bs), dtype='float64')
        self.net[0]['bnz'] = np.zeros((Layers[0], self.bs), dtype='float64')
        self.net[0]['a'] = np.zeros((Layers[0], self.bs), dtype='float64')    
            
        self.net[0]['dJdi'] = np.zeros(self.net[0]['i'].shape, dtype='float64')
        self.net[0]['dJdz'] = np.zeros(self.net[0]['z'].shape, dtype='float64') 
        self.net[0]['dJdbnz'] = np.zeros(self.net[0]['bnz'].shape, dtype='float64')
        self.net[0]['dJda'] = np.zeros(self.net[0]['a'].shape, dtype='float64')
        
        for i in range(1, len(Layers), 1):
            self.net[i]['i'] = np.zeros((Layers[i-1], self.bs), dtype='float64')
            self.net[i]['z'] = np.zeros((Layers[i], self.bs), dtype='float64')
            self.net[i]['bnz'] = np.zeros((Layers[i], self.bs), dtype='float64')
            self.net[i]['a'] = np.zeros((Layers[i], self.bs), dtype='float64')
            self.net[i]['W'] = np.random.randn(Layers[i], Layers[i-1]).astype('float64')
            self.net[i]['b'] = np.random.randn(Layers[i], 1).astype('float64')
            self.net[i]['gamma'] = np.random.randn(Layers[i], 1).astype('float64')
            self.net[i]['beta'] = np.random.randn(Layers[i], 1).astype('float64')
            
            self.net[i]['dJdi'] = np.zeros(self.net[i]['i'].shape, dtype='float64')
            self.net[i]['dJdz'] = np.zeros(self.net[i]['z'].shape, dtype='float64')
            self.net[i]['dJdbnz'] = np.zeros(self.net[i]['bnz'].shape, dtype='float64')
            self.net[i]['dJda'] = np.zeros(self.net[i]['a'].shape, dtype='float64')
            self.net[i]['dJdW'] = np.zeros(self.net[i]['W'].shape, dtype='float64')
            self.net[i]['dJdb'] = np.zeros(self.net[i]['b'].shape, dtype='float64')
            self.net[i]['dJdgamma'] = np.zeros(self.net[i]['gamma'].shape, dtype='float64')
            self.net[i]['dJdbeta'] = np.zeros(self.net[i]['beta'].shape, dtype='float64')
        
        self.p = np.zeros(self.net[-1]['a'].shape, dtype='float64') # Softmax Output
        self.dJdp = np.zeros(self.p.shape, dtype='float64')
        
        self.yhat = np.zeros(self.bs, dtype=int) # Predicted Answer
        self.y_onehot = np.zeros(self.p.shape, dtype=int) # One-Hot-Encoded Predicted Answer
        
        self.J = [] # Loss value trace
        self.J_val = [] # Loss value trace for validation
        self.L2_regularization = [] # L2 regularization value trace
        
    
    # Feed Forward Evaluation
    def forward(self, x):
        
        # For the input layer, Just copy input to Layer 0
        np.copyto(self.net[0]['i'], x)
        np.copyto(self.net[0]['z'], self.net[0]['i'])
        np.copyto(self.net[0]['bnz'], self.net[0]['z'])
        np.copyto(self.net[0]['a'], self.net[0]['bnz'])
        
        for i in range(1, len(self.net)): # Frome Layer 1 to the Last layer
             
            np.copyto(self.net[i]['i'], self.net[i-1]['a']) # Copy last layer's output 
            np.copyto(self.net[i]['z'],
                      np.dot(self.net[i]['W'], self.net[i]['i']) + self.net[i]['b'])           
            np.copyto(self.net[i]['bnz'], 
                      self.batch_norm(self.net[i]['z'],
                                      self.net[i]['gamma'],
                                      self.net[i]['beta'],
                                      0.01))  # 0.01 is epsilon
            np.copyto(self.net[i]['a'], 
                      self.ReLU(self.net[i]['bnz']))# Activation function
            
            
        np.copyto(self.p, self.softmax(self.net[-1]['a']))  # Softmax
        np.copyto(self.yhat, np.argmax(self.p, axis=0)) # Prediction result
        
        return
   
    # Batch Normalization for an MLP Layer
    # x --> bnx
    def batch_norm(self, x, gamma, beta, eps):
        
        bs = x.shape[-1]
        mu = 1.0/bs * np.sum(x, axis=-1)[:, None]
        u = x - mu
        sigma2 = 1.0/bs * np.sum(u ** 2, axis=-1)[:, None]
        q = sigma2 + eps
        v = np.sqrt(q)
        xhat = u / v
        bnx = gamma * xhat + beta
        
        return bnx
    
    # Backprop Batch Normalization for an MLP Layer
    # (dJdbnx, x, gamma, beta) --> (dJdbeta, dJdgamma, dJdx)
    def batch_norm_prime(self, dJdbnx, x, gamma, beta, eps):
        
        bs = x.shape[-1]
        mu = 1.0/bs * np.sum(x, axis=-1)[:, None]
        u = x - mu
        sigma2 = 1.0/bs * np.sum(u ** 2, axis=-1)[:, None]
        q = sigma2 + eps
        v = np.sqrt(q)
        xhat = u / v
        
        dJdbeta = np.mean(dJdbnx, axis=-1)[:, None]
        dJdgamma = np.mean(dJdbnx * xhat, axis=-1)[:, None]
        dJdx = (1.0 - 1.0/bs) * (1.0/v - u**2 / v**3 / bs) * gamma * dJdbnx
        
        return dJdbeta, dJdgamma, dJdx
    
    def softmax(self, a):
        return np.exp(a) / np.sum(np.exp(a), axis=0)
    
    # Sigmoid activation function: z --> a
    def sigmoid(self, z):
        return 1.0 / (1.0 + np.exp(-1.0*z))
    
    # a --> dadz
    def sigmoidPrime(self, a):
        return a * (1.0 - a)
    
    def tanh(self, z):
        return (np.exp(z) - np.exp(-z)) / (np.exp(z) + np.exp(-z))
    
    def tanhPrime(self, a):
        return (1.0 - a ** 2) # Derivative of tanh is (1.0 - tanh ** 2)
    
    def swish(self, z):
        return z * self.sigmoid(z)
    
    def swishPrime(self, a, z):
        return a + self.sigmoid(z) * (1.0 - a) 
    
    def ReLU(self, z):
        a = np.copy(z)
        a[a<0] = 0.0
        return a
    
    def ReLUPrime(self, a):
        dadz = np.copy(a)
        dadz[a>0] = 1.0
        return dadz
    
    def LeakyReLU(self, z, leakyrate):
        a = np.copy(z)
        (rows, cols) = a.shape
        for i in range(rows):
            for j in range(cols):
                a[i, j] = z[i, j] if z[i,j] > 0 else (leakyrate * z[i, j])
        return a
    
    def LeakyReLUPrime(self, a, leakyrate):
        dadz = np.copy(a)
        (rows, cols) = dadz.shape
        for i in range(rows):
            for j in range(cols):
                dadz[i, j] = 1.0 if a[i,j] > 0 else leakyrate
        return dadz
    
    # Loss function
    def loss(self, y, eta):
        
        self.y_onehot.fill(0)
        for i in range(self.bs):
            self.y_onehot[y[i], i] = 1
        self.J.append(-1.0 * np.sum(self.y_onehot * np.log(self.p) / self.bs))
        
        L2 = 0.0 # L2 Regularization
        for i in range(1, len(self.net)):
            L2 += np.sum(self.net[i]['W'] ** 2)              
        self.L2_regularization.append(eta / 2 * L2) # Only the MLP part
        
        return
    
    # Loss function for validation
    def loss_val(self, y, eta):
        
        self.y_onehot.fill(0)
        for i in range(self.bs):
            self.y_onehot[y[i], i] = 1     
        self.J_val.append(-1.0 * np.sum(self.y_onehot * np.log(self.p) / self.bs))
        
        return
            
    # Backward Propagation
    def backprop(self):
        
        self.dJdp = 1.0 / (1.0 - self.y_onehot - self.p)
        dpda = np.array([[self.p[i, :] * (1.0-self.p[j, :]) if i == j
                          else -1 * self.p[i, :] * self.p[j, :]
                          for i in range(self.p.shape[0])]
                         for j in range(self.p.shape[0])])
        for i in range(self.bs):
            self.net[-1]['dJda'][:, i] = np.dot(dpda[:, :, i], self.dJdp[:, i])
            
        for i in range(len(self.net)-1, 0, -1):
            
            np.copyto(self.net[i]['dJdbnz'],
                      (self.net[i]['dJda'] * self.ReLUPrime(self.net[i]['a'])))       
           
            dJdbeta, dJdgamma, dJdBNinput = self.batch_norm_prime(self.net[i]['dJdbnz'],
                                                                  self.net[i]['z'],
                                                                  self.net[i]['gamma'],
                                                                  self.net[i]['beta'],
                                                                  0.01)  
            np.copyto(self.net[i]['dJdbeta'], dJdbeta)
            np.copyto(self.net[i]['dJdgamma'], dJdgamma)
            np.copyto(self.net[i]['dJdz'], dJdBNinput) 
            
            np.copyto(self.net[i]['dJdb'],
                      np.mean(self.net[i]['dJdz'], axis = -1)[:, None])         
            np.copyto(self.net[i]['dJdW'],
                      np.dot(self.net[i]['dJdz'], self.net[i]['i'].T) / self.bs)  
            np.copyto(self.net[i]['dJdi'],
                      np.dot(self.net[i]['W'].T, self.net[i]['dJdz']))
            
            # Copy gradient at the input to be the output gradient of the previous layer
            np.copyto(self.net[i-1]['dJda'], self.net[i]['dJdi'])
        
        # Layer 0 does nothing but passing gradients backward
        np.copyto(self.net[0]['dJdbnz'], self.net[0]['dJda'])
        np.copyto(self.net[0]['dJdz'], self.net[0]['dJdbnz'])
        np.copyto(self.net[0]['dJdi'], self.net[0]['dJdz']) 
        return
    
    # Update parameters
    def update(self, lr, eta):
        
        # Update W, b, gamma, beta with Weight Decay from Layer 1 to the last
        # Layer 0 has no parameters
        for i in range(1, len(self.net)):
            np.copyto(self.net[i]['W'],
                      (1.0 - eta * lr) * self.net[i]['W'] - lr*self.net[i]['dJdW'])
            np.copyto(self.net[i]['b'],
                      (1.0 - eta * lr) * self.net[i]['b'] - lr*self.net[i]['dJdb'])
            np.copyto(self.net[i]['gamma'],
                      (1.0 - eta * lr) * self.net[i]['gamma'] - lr*self.net[i]['dJdgamma'])
            np.copyto(self.net[i]['beta'],
                      (1.0 - eta * lr) * self.net[i]['beta'] - lr*self.net[i]['dJdbeta'])
        return
    
    # Train MLP alone. For a CNN, training is via the CNN instance
    def train(self, train_x, train_y, epoch_count, lr, eta):
        
        for e in range(epoch_count):
            # print ("Epoch ", e)
            for i in range(train_x.shape[1]//self.bs):
                x = train_x[:, i*self.bs:(i+1)*self.bs]
                y = train_y[i*self.bs:(i+1)*self.bs]
                self.forward(x)
                self.loss(y, eta)
                self.backprop()
                self.update(lr, eta)           
        return  
    
    

# CNN function

In [3]:
class Conv_Layer(object):
    '''Convolution Layer including 2D convolution, pooling, activation and batch normalization'''
    def __init__(self,
                 bs,    #batch size
                 i_ch,  #input channel
                 i_h,  #input height
                 i_w,  #input weight
                 k_num, # k_num == o_ch; number of kernels equal number of output channels
                 k_h,   #kernel height
                 k_w,   #kernel weight
                 stride = 1, # Stride
                 zp = 0, # Zero Padding
                 mph = 2, # Pooling kernel height
                 mpw = 2, # Pooling kernel width
                 leakyrate = 0.1):
        
        #convert to layer
        self.bs = bs
        self.i_ch = i_ch
        self.i_h = i_h
        self.i_w = i_w
        self.k_num = k_num
        self.k_h = k_h
        self.k_w = k_w
        self.stride = stride
        self.zp = zp
        self.mph = mph
        self.mpw = mpw
        self.leakyrate = leakyrate
       
        # conv2d(ai, W, b) --> z
        # activation(z) --> zr
        # Batch_norm(zr, gamma, beta) --> r
        # Pool(r) --> ao, mp_trace
        self.ai = np.zeros((bs, i_ch, i_h+2*zp, i_w+2*zp), dtype='float64')
        self.z = np.zeros((bs, k_num, (i_h - k_h+1)//stride, (i_w - k_w+1)//stride), dtype='float64')
        self.bnz = np.zeros(self.z.shape, dtype='float64')
        self.r = np.zeros(self.z.shape, dtype='float64')
        self.mp_trace = np.zeros(self.r.shape, dtype='int32')
        self.ao = np.zeros((bs, k_num, (i_h - k_h+1)//stride//mph, (i_w - k_w+1)//stride//mpw), dtype='float64')
        self.W = np.random.randn(k_num, i_ch, k_h, k_w).astype('float64')
        self.b = np.random.randn(k_num, 1).astype('float64')
        self.gamma = np.ones(self.z.shape[1:], dtype='float64')
        self.beta = np.zeros(self.z.shape[1:], dtype='float64')
        
        self.dJdai = np.zeros(self.ai.shape, dtype='float64')
        self.dJdz = np.zeros(self.z.shape, dtype='float64')
        self.dJdbnz = np.zeros(self.bnz.shape, dtype='float64')
        self.dJdr = np.zeros(self.r.shape, dtype='float64')
        self.dJdao = np.zeros(self.ao.shape, dtype='float64')
        self.dJdW = np.zeros(self.W.shape, dtype='float64')
        self.dJdb = np.zeros(self.b.shape, dtype='float64')
        self.dJdgamma = np.zeros(self.gamma.shape, dtype='float64')
        self.dJdbeta = np.zeros(self.beta.shape, dtype='float64')
        
    def conv2d(self):  # (ai, W, b) --> z
        
        (i_bs, i_ch, i_rows, i_cols) = self.ai.shape # Input data
        (k_num, k_ch, k_rows, k_cols) = self.W.shape # Kernels
        (z_bs, z_ch, z_rows, z_cols) = self.z.shape   # Output z_ch == k_num
        # self.ai.fill(0.0)
        # np.copyto(self.ai[:, :, self.zp:self.zp+i_rows, self.zp:self.zp+i_cols], i_data)
        s = self.stride
        
        for b in range(z_bs): # Step over mini batch
            for c in range(z_ch): # z_ch == k_num
                for i in range(z_rows):
                    for j in range(z_cols): 
                        self.z[b, c, i, j] = np.sum(np.multiply(self.W[c, :,  :, :],   #multipy two matrix
                                                                self.ai[b,
                                                                        :,
                                                                        i*s : i*s+k_rows,
                                                                        j*s : j*s+k_cols])
                                                   ) + self.b[c]
        return 
        
    
    def conv2d_prime(self): # (dJdz, W, ai) --> (dJdai, dJdW, dJdb)
        (z_bs, z_ch, z_rows, z_cols) = self.dJdz.shape
        (a_bs, a_ch, a_rows, a_cols) = self.dJdai.shape
        (k_num, k_ch, k_rows, k_cols) = self.dJdW.shape
        (b_num, b_ch) = self.dJdb.shape
        s = self.stride # self.stride
        self.dJdai.fill(0.0)
        self.dJdW.fill(0.0)
        self.dJdb.fill(0.0)
        
        # For each single dJdZ value, add a kernels to dJdai
        for b in range(z_bs):
            for c in range(z_ch):
                for i in range(z_rows):
                    for j in range(z_cols):
                        dJdz_value = self.dJdz[b, c, i, j]
                        self.dJdai[b, :, i*s:i*s+k_rows, j*s:j*s+k_cols] += dJdz_value * self.W[c, :, :, :]
        
        # Each dJdW value is a dot product of two arrays              
        for k in range(k_num):
            for c in range(k_ch):
                for i in range(k_rows):
                    for j in range(k_cols):
                        self.dJdW[k, c, i, j] = np.sum(self.ai[:, c, i:i+z_rows*s:s, j:j+z_cols*s:s] *
                                                       self.dJdz[:, k, :, :]) / self.bs 
        
        # Each dJdb value is a sum of a whole dJdZ array
        for c in range(z_ch):
            self.dJdb[c] = np.sum(self.dJdz[:, c, :, :]) / self.bs
            
        return 

    # All activation functions produce  z --> zr  
    def LeakyReLU(self): 
        np.copyto(self.r, np.where(self.z > 0, 1.0 * self.bnz, self.leakyrate * self.bnz))
        return
    
    def LeakyReLU_prime(self):
        np.copyto(self.dJdbnz, np.where(self.z > 0, 1.0 * self.dJdr, self.leakyrate * self.dJdr))
        return
    
    def tanh(self):
        np.copyto(self.r, (np.exp(self.zr) - np.exp(-self.zr)) / (np.exp(self.zr) + np.exp(-self.zr)))
        return
    
    def tanh_prime(self):
        np.copyto(self.dJdzr, (1.0 - self.r ** 2))
        return
    
    def swish(self):
        np.copyto(self.zr, self.z * self.sigmoid(self.z))
        return
    
    def swish_prime(self):
        np.copyto(self.dJdz,
                  self.zr + self.sigmoid(self.z) * (1.0 - self.zr))
        return
    
    def sigmoid(self, z):
        return 1.0 / (1.0 + np.exp(-z))
    
    # Batch normalization
    # (zr, gamma, beta) --> r
    def batch_norm(self, eps=0.01):  
        
        x = self.z
        bs = x.shape[0]
        mu = 1.0/bs * np.sum(x, axis=0)[None, :, :, :]
        u = x - mu
        sigma2 = 1.0/bs * np.sum(u ** 2, axis=0)[None, :, :, :]
        q = sigma2 + eps
        v = np.sqrt(q)
        xhat = u / v
        
        np.copyto(self.bnz,
                  self.gamma * xhat + self.beta)
        
        return
    
    # (dJdr, zr, gamma) --> (dJdbeta, dJdgamma, dJdzr)
    def batch_norm_prime(self, eps = 0.01):
        
        x = self.z
        bs = x.shape[0]
        mu = 1.0/bs * np.sum(x, axis=0)[None, :, :, :]
        u = x - mu
        sigma2 = 1.0/bs * np.sum(u ** 2, axis=0)[None, :, :, :]
        q = sigma2 + eps
        v = np.sqrt(q)
        xhat = u / v
        
        self.dJdbeta = np.mean(self.dJdbnz, axis=0)[None, :, :, :]
        self.dJdgamma = np.mean(self.dJdbnz * xhat, axis=0)[None, :, :, :]
        self.dJdz = (1.0 - 1.0/bs) * (1.0/v - u**2/v**3/bs) * self.gamma * self.dJdbnz
        
        return 
    
    def batch_norm_pass(self, eps=0.01):
        
        np.copyto(self.r, self.zr)
        
        return
    
    def batch_norm_prime_pass(self, eps = 0.01):
        
        np.copyto(self.dJdzr, self.dJdr)     
        return   
    
    # Maximum Pooling
    # r --> (ao, mp_trace)
    def max_pool(self):  
        (r_bs, r_ch, r_rows, r_cols) = self.r.shape
        (a_bs, a_ch, a_rows, a_cols) = self.ao.shape
        self.mp_trace.fill(0) # record from which is the maximum so we can backprop
        
        for b in range(a_bs):
            for c in range(a_ch):
                for i in range(a_rows):
                    for j in range(a_cols):
                        pool_src2d = self.r[b,
                                            c,
                                            i*self.mph:(i+1)*self.mph,
                                            j*self.mpw:(j+1)*self.mpw] 
                        self.ao[b, c, i, j] = np.max(pool_src2d)
                        max_pos = np.unravel_index(np.argmax(pool_src2d), 
                                                   np.shape(pool_src2d))
                        self.mp_trace[b, c, i*self.mph+max_pos[0], j*self.mpw+max_pos[1]] = 1
        return
    
    def max_pool_prime(self):
        (r_bs, r_ch, r_rows, r_cols) = self.dJdr.shape
        
        for b in range(r_bs):
            for c in range(r_ch):
                for i in range(r_rows):
                    for j in range(r_cols):
                        self.dJdr[b, c, i, j] = (self.dJdao[b, c, i//self.mph, j//self.mpw] 
                                                  if self.mp_trace[b, c, i, j] == 1
                                                  else 0.0)                       
        return
    
    def forward(self):
        
        self.conv2d()  # ai --> z
        self.batch_norm()  # z --> bnz
        self.LeakyReLU()  # bnz --> r
        self.max_pool()  # r --> ao
        return
    
    def backprop(self):
        self.max_pool_prime() # dJdao --> dJdr
        self.LeakyReLU_prime() # dJdr --> dJdbnz
        self.batch_norm_prime() # dJdnz --> dJdz
        self.conv2d_prime() # dJdz --> dJdai
        
        return
    
    def update(self, lr, eta):
        # Update W, b, gamma, beta with Weight Decay
        self.W = (1.0 - eta * lr) * self.W - lr * self.dJdW
        self.b = (1.0 - eta * lr) * self.b - lr * self.dJdb
        self.gamma = (1.0 - eta * lr) * self.gamma - lr * self.dJdgamma
        self.beta = (1.0 - eta * lr) * self.beta - lr * self.beta
        return
    
   

In [4]:
# A Convolutional Neural Network (CNN) Consists of a number of conv layers followed by
# a number of fully-connected layers
class CNN(object):
    def __init__(self,
                input_data_spec = [10, 3, 28, 28],
                conv_layer_spec = [{"k_num" : 2,
                                    "k_h" : 3,
                                    "k_w" : 3,
                                    "stride" : 1,
                                    "zp" : 0,
                                    "mph" : 2,
                                    "mpw" :2}],
                 fc_layer_spec = [100, 50, 10]):
        
        # A list of conv layers
        self.c_net = []
        for i in range(len(conv_layer_spec)):
            self.c_net.append(Conv_Layer(bs = input_data_spec[0],
                                  i_ch = input_data_spec[1],
                                  i_h = input_data_spec[2],
                                  i_w = input_data_spec[3],
                                  k_num = conv_layer_spec[i]['k_num'],
                                  k_h = conv_layer_spec[i]['k_h'],
                                  k_w = conv_layer_spec[i]['k_w'],
                                  stride = conv_layer_spec[i]['stride'],
                                  zp = conv_layer_spec[i]['zp'],
                                  mph = conv_layer_spec[i]['mph'],
                                  mpw = conv_layer_spec[i]['mpw']))
            input_data_spec = list(self.c_net[i].ao.shape)
                       
        (bs, ch, r, c) = self.c_net[-1].ao.shape # Shape of Last Conv Layer's Output Feature Map
        fc_layer_spec[0] = ch*r*c
        self.fc_net = MLP(fc_layer_spec, BatchSize=input_data_spec[0])
        return
    
    def forward(self, input_data):
        
        # Forward through Conv Layers
        for i in range(len(self.c_net)):
            if i == 0:
                np.copyto(self.c_net[i].ai, input_data)
            else:
                np.copyto(self.c_net[i].ai, self.c_net[i-1].ao)
            self.c_net[i].forward()
            
        # Flatten the last output feature map of the conv layer for feeding to the MLP    
        input_to_fc = np.copy(self.c_net[-1].ao.reshape(self.c_net[-1].ao.shape[0], -1).T)
        
        # Forward through the MLP ( anumber of fully connected layers
        self.fc_net.forward(input_to_fc)
        
        return
    
    def backprop(self):
        
        # Backprop MLP
        self.fc_net.backprop()
        
        # Transfer the gradients from the MLB to the conv layers
        np.copyto(self.c_net[-1].dJdao, 
                  self.fc_net.net[0]['dJdi'].T.reshape(self.c_net[-1].dJdao.shape))
        
        # Backprop Conv layers
        for i in range(len(self.c_net)-1, -1, -1):
            self.c_net[i].backprop()
            if i > 0:
                np.copyto(self.c_net[i-1].dJdao, self.c_net[i].dJdai)
        return
    
    def update(self, lr, eta):
        
        # Weight update of conv layers
        for i in range(len(self.c_net)):
            self.c_net[i].update(lr, eta)
            
        # Weight Update of MLP
        self.fc_net.update(lr, eta)
        
        return

    # Find learning rate
    def proper_lr(self):
        max_W = 0.0
        max_dJdW = 0.0
        for conv_net in self.c_net:
            max_W = max(max_W, np.max(np.abs(conv_net.W)))
            max_dJdW = max(max_dJdW, np.max(np.abs(conv_net.dJdW)))
        for fc_net in self.fc_net.net[1:]:
            max_W = max(max_W, np.max(np.abs(fc_net['W'])))
            max_dJdW = max(max_dJdW, np.max(np.abs(fc_net['dJdW'])))
        return min(1.0, 0.1 * max_W / max_dJdW)
    

        
    
    # Train a CNN end-to-end
    def train(self, train_x, train_y, val_x, val_y, epoch_count, lr, eta):
        
        for e in range(epoch_count):
            # Randomly shuffle the training data at the begining of an epoch
            shuffle = np.arange(train_x.shape[0])
            np.random.shuffle(shuffle)
            train_x_s = train_x[shuffle]
            train_y_s = train_y[shuffle]
            bs = self.fc_net.bs
            
            for i in range(train_x_s.shape[0]//bs):
                x = train_x_s[i*bs:(i+1)*bs, :, : ,:]
                y = train_y_s[i*bs:(i+1)*bs]    
                self.forward(x)
                self.fc_net.loss(y, eta)
                self.backprop()
                self.update(lr, eta)
                
                print ("\nEpoch", e, "Batch", i, "J=", self.fc_net.J[-1], 
                       "Error Rate=", np.count_nonzero(np.array(y-self.fc_net.yhat)) / float(len(y)))
                      # "\nyhat=", self.fc_net.yhat, "\n   y=", y)
                #print ("\nProper lr=", lr)
                print ("\nMax abs(a) of Last MLP Layer=", np.max(np.abs(self.fc_net.net[-1]['a'])), 
                       "\nMax abs(dJda) of Last MLP Layer=", np.max(np.abs(self.fc_net.net[-1]['dJda'])),
                      "\nMax abs(W) of Last MLP Layer=", np.max(np.abs(self.fc_net.net[1]['W'])), "\n")
            # Validation
            for i in range(val_x.shape[0]//bs):
                x = val_x[i*bs:(i+1)*bs, :, : ,:]
                y = val_y[i*bs:(i+1)*bs]    
                self.forward(x)
                self.fc_net.loss_val(y, eta)
                
        return             

# load data for MLP

In [15]:
f = open("mnist_train_60000.csv", 'r')
a = f.readlines()
f.close()

# f = figure(figsize=(15,15));
%matplotlib inline
import matplotlib.pylab as plt
x = []
y = []
count=1
for line in a:
    linebits = line.split(',')
    x_line = [int(linebits[i]) for i in range(len(linebits))]
    x.append(x_line[1:])
    y.append(x_line[0])
    imarray = np.asfarray(linebits[1:]).reshape((28,28))
    count += 1

    pass


train_60000_x_MLP = np.array(x).T
train_60000_y_MLP = np.array(y)

In [21]:
Layers = (784, 50, 10)
BatchSize = 60000
EpochCount = 100
LearningRate = 0.1
Eta = 0.00001

mlp60000 = MLP(Layers, BatchSize)


# Run MLP

In [31]:
mlp60000.train(train_60000_x_MLP, train_60000_y_MLP, EpochCount*2, LearningRate, Eta)

# Get accuracy

In [18]:
def acc(target,predict):
    correct = 0
    for i in range(len(target)):
        if (target[i] - predict[i]) == 0:
            correct += 1
    return correct/len(target)

In [32]:
print("Accuracy = ", acc(train_60000_y_MLP, mlp60000.yhat))

Accuracy =  0.5819166666666666


# load data for CNN

In [5]:
# Read in 60000 digits


f = open("mnist_train_60000.csv", 'r')
a = f.readlines()
f.close()

# f = figure(figsize=(15,15));
%matplotlib inline
import matplotlib.pylab as plt
x = []
y = []
count=1
for line in a:
    linebits = line.split(',')
    x_line = [int(linebits[i]) for i in range(len(linebits))]
    x.append(x_line[1:])
    y.append(x_line[0])
    
    pass


train_60000_x = np.clip(np.array(x), 0, 1).reshape(60000, 1, 28, 28)
train_60000_y = np.array(y)


Run time is     13.17792    seconds


# Run CNN

In [10]:
# Build a CNN identical to LeNet
BatchSize = 250
EpochCount = 1
LearningRate = 0.01
Eta = 0.001

input_data_spec = [BatchSize, 1, 28, 28] # [BatchSize, number of input chanels, Width, Height]
conv_layer_spec = [{"k_num" : 6, "k_h" : 5, "k_w" : 5, "stride" : 1, "zp" : 0, "mph" : 2,"mpw" :2},
                   {"k_num" : 16, "k_h" : 3, "k_w" : 3, "stride" : 1, "zp" : 0, "mph" : 2,"mpw" :2}]
fc_layer_spec = [128,128,128,10]

cnn60000 = CNN(input_data_spec=input_data_spec, 
          conv_layer_spec=conv_layer_spec, 
          fc_layer_spec=fc_layer_spec)


In [13]:
start_time = time.clock()
cnn60000.train(train_60000_x, train_60000_y,train_60000_x, train_60000_y, EpochCount, LearningRate, Eta)
print("Run time for is    ",time.clock() - start_time, "   seconds")


Epoch 0 Batch 0 J= 2.2932751539117557 Error Rate= 0.748

Max abs(a) of Last MLP Layer= 5.969989157424951 
Max abs(dJda) of Last MLP Layer= 1.8706379846284926 
Max abs(W) of Last MLP Layer= 4.892952844692668 


Epoch 0 Batch 1 J= 2.308966052643724 Error Rate= 0.776

Max abs(a) of Last MLP Layer= 5.370730981686749 
Max abs(dJda) of Last MLP Layer= 1.691641809786176 
Max abs(W) of Last MLP Layer= 4.892900423346518 


Epoch 0 Batch 2 J= 2.2010429346513503 Error Rate= 0.72

Max abs(a) of Last MLP Layer= 5.2643205738776055 
Max abs(dJda) of Last MLP Layer= 1.7030184317995802 
Max abs(W) of Last MLP Layer= 4.89285099040263 


Epoch 0 Batch 3 J= 2.2153309843165836 Error Rate= 0.708

Max abs(a) of Last MLP Layer= 5.400514910556581 
Max abs(dJda) of Last MLP Layer= 1.5194634523368546 
Max abs(W) of Last MLP Layer= 4.892793234295298 


Epoch 0 Batch 4 J= 2.219598594676271 Error Rate= 0.72

Max abs(a) of Last MLP Layer= 5.166658149269281 
Max abs(dJda) of Last MLP Layer= 1.7206444316966119 
Max a


Epoch 0 Batch 40 J= 2.1860921515616205 Error Rate= 0.688

Max abs(a) of Last MLP Layer= 4.854820072143543 
Max abs(dJda) of Last MLP Layer= 1.5626770786857564 
Max abs(W) of Last MLP Layer= 4.89057357329037 


Epoch 0 Batch 41 J= 2.1151058675234378 Error Rate= 0.7

Max abs(a) of Last MLP Layer= 5.117689380347505 
Max abs(dJda) of Last MLP Layer= 1.3439824887092873 
Max abs(W) of Last MLP Layer= 4.890523687221976 


Epoch 0 Batch 42 J= 2.1614408262687794 Error Rate= 0.696

Max abs(a) of Last MLP Layer= 5.313895694114334 
Max abs(dJda) of Last MLP Layer= 1.5273935982153648 
Max abs(W) of Last MLP Layer= 4.890467409205957 


Epoch 0 Batch 43 J= 2.1494794240095523 Error Rate= 0.688

Max abs(a) of Last MLP Layer= 4.614997209256595 
Max abs(dJda) of Last MLP Layer= 1.4199652639731233 
Max abs(W) of Last MLP Layer= 4.890405625137235 


Epoch 0 Batch 44 J= 2.095143566430158 Error Rate= 0.66

Max abs(a) of Last MLP Layer= 5.330965404801272 
Max abs(dJda) of Last MLP Layer= 1.3666762676990878 



Epoch 0 Batch 79 J= 2.191383617312427 Error Rate= 0.7

Max abs(a) of Last MLP Layer= 6.596315340463782 
Max abs(dJda) of Last MLP Layer= 1.7466938918369745 
Max abs(W) of Last MLP Layer= 4.88826097954313 


Epoch 0 Batch 80 J= 2.2172552210979473 Error Rate= 0.74

Max abs(a) of Last MLP Layer= 5.697065692434295 
Max abs(dJda) of Last MLP Layer= 1.4424074752273792 
Max abs(W) of Last MLP Layer= 4.8881992012560636 


Epoch 0 Batch 81 J= 2.3173062192028877 Error Rate= 0.752

Max abs(a) of Last MLP Layer= 5.773497940290929 
Max abs(dJda) of Last MLP Layer= 1.396257457743042 
Max abs(W) of Last MLP Layer= 4.888145105713163 


Epoch 0 Batch 82 J= 2.273510727289783 Error Rate= 0.732

Max abs(a) of Last MLP Layer= 5.225684455759162 
Max abs(dJda) of Last MLP Layer= 1.5187529314528225 
Max abs(W) of Last MLP Layer= 4.888093954610187 


Epoch 0 Batch 83 J= 2.1420769043266974 Error Rate= 0.708

Max abs(a) of Last MLP Layer= 5.354804751463109 
Max abs(dJda) of Last MLP Layer= 1.6361101884830687 
M


Epoch 0 Batch 118 J= 2.1717713182395606 Error Rate= 0.7

Max abs(a) of Last MLP Layer= 5.451289669289849 
Max abs(dJda) of Last MLP Layer= 1.5486189037620157 
Max abs(W) of Last MLP Layer= 4.885989727136165 


Epoch 0 Batch 119 J= 2.1158391975945317 Error Rate= 0.696

Max abs(a) of Last MLP Layer= 5.369222497861305 
Max abs(dJda) of Last MLP Layer= 1.3704575754903612 
Max abs(W) of Last MLP Layer= 4.885933632969744 


Epoch 0 Batch 120 J= 2.1619475454329096 Error Rate= 0.732

Max abs(a) of Last MLP Layer= 5.602864088096582 
Max abs(dJda) of Last MLP Layer= 1.6113579533160796 
Max abs(W) of Last MLP Layer= 4.885868875372322 


Epoch 0 Batch 121 J= 2.1598389668084854 Error Rate= 0.712

Max abs(a) of Last MLP Layer= 5.647755914861606 
Max abs(dJda) of Last MLP Layer= 1.4753495998185304 
Max abs(W) of Last MLP Layer= 4.885811631925684 


Epoch 0 Batch 122 J= 2.1249291344458094 Error Rate= 0.724

Max abs(a) of Last MLP Layer= 5.042797142209571 
Max abs(dJda) of Last MLP Layer= 1.3720327591


Epoch 0 Batch 157 J= 2.203310035559638 Error Rate= 0.748

Max abs(a) of Last MLP Layer= 5.915624539551208 
Max abs(dJda) of Last MLP Layer= 1.633506331391922 
Max abs(W) of Last MLP Layer= 4.88364653021209 


Epoch 0 Batch 158 J= 2.1088217040930894 Error Rate= 0.7

Max abs(a) of Last MLP Layer= 5.436676778536992 
Max abs(dJda) of Last MLP Layer= 1.1990788162453452 
Max abs(W) of Last MLP Layer= 4.88359618001176 


Epoch 0 Batch 159 J= 2.282842365193745 Error Rate= 0.784

Max abs(a) of Last MLP Layer= 5.725318877619375 
Max abs(dJda) of Last MLP Layer= 1.7714420981744947 
Max abs(W) of Last MLP Layer= 4.883538100142895 


Epoch 0 Batch 160 J= 2.086042058914206 Error Rate= 0.688

Max abs(a) of Last MLP Layer= 5.133237794755155 
Max abs(dJda) of Last MLP Layer= 1.2063441122273044 
Max abs(W) of Last MLP Layer= 4.883479626282704 


Epoch 0 Batch 161 J= 2.1599568028104867 Error Rate= 0.712

Max abs(a) of Last MLP Layer= 6.103496538639799 
Max abs(dJda) of Last MLP Layer= 1.7427751081448424


Epoch 0 Batch 196 J= 2.091182987865965 Error Rate= 0.692

Max abs(a) of Last MLP Layer= 5.258114321412341 
Max abs(dJda) of Last MLP Layer= 1.8182821408742882 
Max abs(W) of Last MLP Layer= 4.88136195100478 


Epoch 0 Batch 197 J= 2.1597682274064978 Error Rate= 0.724

Max abs(a) of Last MLP Layer= 6.188150200513139 
Max abs(dJda) of Last MLP Layer= 1.7285297007687732 
Max abs(W) of Last MLP Layer= 4.881298256726095 


Epoch 0 Batch 198 J= 2.105679269159176 Error Rate= 0.7

Max abs(a) of Last MLP Layer= 6.485381801021719 
Max abs(dJda) of Last MLP Layer= 1.2068686608780923 
Max abs(W) of Last MLP Layer= 4.881238339183915 


Epoch 0 Batch 199 J= 2.10952126607697 Error Rate= 0.692

Max abs(a) of Last MLP Layer= 5.2736248927048734 
Max abs(dJda) of Last MLP Layer= 1.235411762903548 
Max abs(W) of Last MLP Layer= 4.881178229906412 


Epoch 0 Batch 200 J= 2.193575297112867 Error Rate= 0.716

Max abs(a) of Last MLP Layer= 5.331199190728458 
Max abs(dJda) of Last MLP Layer= 1.4918761588925773


Epoch 0 Batch 235 J= 2.0143210021057816 Error Rate= 0.676

Max abs(a) of Last MLP Layer= 5.9480449020106665 
Max abs(dJda) of Last MLP Layer= 1.2684976780257982 
Max abs(W) of Last MLP Layer= 4.879025523184017 


Epoch 0 Batch 236 J= 2.005959435737979 Error Rate= 0.632

Max abs(a) of Last MLP Layer= 6.0876190150551635 
Max abs(dJda) of Last MLP Layer= 1.5297461177445228 
Max abs(W) of Last MLP Layer= 4.878962659203254 


Epoch 0 Batch 237 J= 2.080485762506192 Error Rate= 0.692

Max abs(a) of Last MLP Layer= 6.745360294179294 
Max abs(dJda) of Last MLP Layer= 1.389909940838945 
Max abs(W) of Last MLP Layer= 4.87890002004301 


Epoch 0 Batch 238 J= 2.0784134947112745 Error Rate= 0.708

Max abs(a) of Last MLP Layer= 5.856341632922945 
Max abs(dJda) of Last MLP Layer= 1.0918100627834826 
Max abs(W) of Last MLP Layer= 4.878838360152667 


Epoch 0 Batch 239 J= 2.041884449971188 Error Rate= 0.668

Max abs(a) of Last MLP Layer= 5.363665053737062 
Max abs(dJda) of Last MLP Layer= 1.22885185594

KeyboardInterrupt: 

# CNN result : error = ~0.65